## Processin SIM data

In [1]:
from pysus.online_data import SIM, parquets_to_dataframe
from pysus.preprocessing.decoders import translate_variables_SIM
from pysus.preprocessing.SIM import group_and_count, redistribute_missing, redistribute_cid_chapter

In [2]:
df = parquets_to_dataframe(SIM.download('ac',2010))
df

,contador,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,DTNASC,IDADE,SEXO,RACACOR,...,DTCADASTRO,ATESTANTE,FONTEINV,DTRECEBIM,UFINFORM,CB_PRE,MORTEPARTO,DTCADINF,TPOBITOCOR,DTCADINV
0,1,1,2,06082010,2000,831,09041945,465,2,,...,19082010,,,20092010,,R98,,,,
1,2,1,2,06082010,1300,812,20011912,498,1,,...,19082010,,2,01102010,,R98,,,,
2,3,1,2,02102010,1700,812,17032010,306,2,4,...,05012011,3,,10022011,,R98,,,,
3,4,1,2,07042010,2300,,07042010,005,2,1,...,06052010,1,,20072010,,O689,,,,
4,5,1,2,13052010,0030,812,04081971,438,1,1,...,21062010,5,,20072010,,X999,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3004,3005,1,2,28092010,0100,,20041958,452,2,4,...,20102010,,,11112010,,I619,,,,
3005,3006,1,2,14102010,1015,,24031928,482,1,1,...,06122010,2,,19012011,,I219,,,,
3006,3007,1,2,21112010,0650,,11121955,454,2,1,...,09122010,5,,17012011,,C349,,,,
3007,3008,1,2,10112010,0500,,07071955,455,1,1,...,04012011,2,,19012011,,C787,,,,


In [3]:
df['SEXO'] = df['SEXO'].astype('object')

In [4]:
variables = ['CODMUNRES','SEXO','IDADE_ANOS','CID10_CHAPTER']

df = translate_variables_SIM(
                                df,
                                age_classes=True,
                                classify_cid10_chapters=True,
                            )
df = df[variables]


2023-04-11 14:08:10.623 | DEBUG    | pysus.online_data.SIM:get_municipios:180 - Stablishing connection with ftp.datasus.gov.br.
220 Microsoft FTP Service
2023-04-11 14:08:10.659 | DEBUG    | pysus.online_data.SIM:get_municipios:184 - Changing FTP work dir to: /dissemin/publicos/SIM/CID10/TABELAS
2023-04-11 14:08:10.661 | INFO     | pysus.online_data.SIM:get_municipios:194 - Local parquet file found at /home/luabida/pysus/SIM_CADMUN_.parquet
/home/luabida/Projetos/InfoDengue/PySUS/pysus/preprocessing/decoders.py:122: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return df["MUNCODDV"].append(df["MUNCOD"]).astype("int64").values
2023-04-11 14:08:10.920 | DEBUG    | pysus.online_data.SIM:get_CID10_chapters_table:42 - Stablishing connection with ftp.datasus.gov.br.
220 Microsoft FTP Service
2023-04-11 14:08:10.955 | DEBUG    | pysus.online_data.SIM:get_CID10_chapters_table:46 - Changing FTP work dir to

In [5]:
counts = group_and_count(df,variables)
counts["COUNTS_ORIGINAL"] = counts["COUNTS"]
nan_string = 'missing'

In [6]:
counts = redistribute_missing(counts,['CODMUNRES','SEXO','IDADE_ANOS'],nan_string=nan_string)
counts[counts["COUNTS"] > 1]

,CODMUNRES,SEXO,IDADE_ANOS,CID10_CHAPTER,COUNTS,COUNTS_ORIGINAL
13,1200013,Feminino,"[0.0, 1.0)",16,2.0,2.0
9397,1200104,Feminino,"[0.0, 1.0)",16,2.0,2.0
10615,1200104,Feminino,"[72.0, 73.0)",10,2.0,2.0
10961,1200104,Masculino,"[0.0, 1.0)",16,3.0,3.0
11423,1200104,Masculino,"[27.0, 28.0)",20,2.0,2.0
...,...,...,...,...,...,...
96787,1200708,Masculino,"[81.0, 82.0)",9,2.0,2.0
98539,1200807,Feminino,"[0.0, 1.0)",10,2.0,2.0
98545,1200807,Feminino,"[0.0, 1.0)",16,4.0,4.0
101420,1200807,Masculino,"[77.0, 78.0)",18,2.0,2.0


In [7]:
counts = redistribute_cid_chapter(counts,['CODMUNRES','SEXO','IDADE_ANOS'])
counts[counts['CID10_CHAPTER'] == 1]

,CODMUNRES,SEXO,IDADE_ANOS,CID10_CHAPTER,COUNTS,COUNTS_ORIGINAL
0,1200013,Feminino,"[0.0, 1.0)",1,1.0,1.0
17,1200013,Feminino,"[1.0, 2.0)",1,0.0,0.0
34,1200013,Feminino,"[2.0, 3.0)",1,0.0,0.0
51,1200013,Feminino,"[3.0, 4.0)",1,0.0,0.0
68,1200013,Feminino,"[4.0, 5.0)",1,0.0,0.0
...,...,...,...,...,...,...
107831,NA,NA,"[87.0, 88.0)",1,0.0,0.0
107848,NA,NA,"[88.0, 89.0)",1,0.0,0.0
107865,NA,NA,"[89.0, 90.0)",1,0.0,0.0
107882,NA,NA,"[90.0, inf)",1,0.0,0.0
